In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

In [13]:
ratings_df = pd.read_csv('../data/ml-latest-small/ratings.csv')
R = ratings_df.pivot(index='userId', columns='movieId', values='rating')
R.tail()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
R_imputed = R.fillna(R.mean())

In [40]:
model = NMF(n_components=20, init='random', random_state=15, max_iter=1000, alpha=5)

model.fit(R_imputed)

Q = model.components_  

P = model.transform(R_imputed)  

print(model.reconstruction_err_)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 1000 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


242.10561092416148


In [41]:
nR = np.dot(P,Q)
nR

array([[4.20446604, 3.71214027, 3.56531085, ..., 3.49010236, 3.49922168,
        4.01205247],
       [3.88908768, 3.44041401, 3.23749722, ..., 3.49245798, 3.49604643,
        4.00045515],
       [3.81650905, 3.44477349, 3.23775001, ..., 3.49831578, 3.49745438,
        3.99586749],
       ...,
       [2.50822558, 2.38573424, 2.15578061, ..., 3.51893883, 3.50693313,
        3.99041219],
       [3.83212311, 3.39735855, 3.20566805, ..., 3.49081391, 3.49541575,
        4.00114371],
       [4.22483081, 3.80964338, 3.13526459, ..., 3.49682128, 3.5064316 ,
        4.02061158]])

In [42]:
def get_unseen_films(user_id):
    users_unseen_films = R.T.isna()
    user_boolean_series = users_unseen_films[[user_id]]
    unseen_films = user_boolean_series.index[user_boolean_series[user_id]].tolist()
    return unseen_films

In [45]:
def nmf_recommender(user_id, k, get_unseen_films):
    """
    return k highest rated unseen movies as predicted by non negative matrix factorization
    """
    ratings_user = R_imputed.loc[[user_id]]
    query = ratings_user.values
    
    p_user = model.transform(query)
    r_hat_user = model.inverse_transform(p_user)
    
    r_hat_df = pd.DataFrame(r_hat_user)
    r_hat_df = r_hat_df.T      
    
    not_seen_sorted = r_hat_df[r_hat_df.index.isin(get_unseen_films(user_id))].sort_values(by=0,ascending=False)
        
    return not_seen_sorted[:k].index.tolist()

In [46]:
nmf_recommender(5,7,get_unseen_films)

[2745, 7810, 8943, 4103, 4384, 8928, 5015]